In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import warnings
from PIL import Image
warnings.filterwarnings('ignore')

In [2]:
path_original = '256X'
path_blurred = 'Zgaussowane'

# Funkcja do ładowania i przetwarzania obrazów
def load_images(path, size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(path)):
        img_path = os.path.join(path, filename)
        img = load_img(img_path, target_size=size, color_mode='rgb')
        img = img_to_array(img) / 255.0  # Normalizacja obrazów do zakresu [0, 1]
        images.append(img)
    return np.array(images)

In [3]:
original_images = load_images(path_original)
blurred_images = load_images(path_blurred)

print(f'Liczba załadowanych oryginalnych obrazów: {len(original_images)}')
print(f'Liczba załadowanych zblurowanych obrazów: {len(blurred_images)}')

Liczba załadowanych oryginalnych obrazów: 57
Liczba załadowanych zblurowanych obrazów: 57


In [10]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    # model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    # odel.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

input_shape = (256, 256, 3)
model = build_model(input_shape)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 256, 256, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 256, 256, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,683 (1.28 MB)

 Trainable params: 335,683 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from sklearn.model_selection import train_test_split

# Podział danych na treningowe i walidacyjne
X_train, X_val, y_train, y_val = train_test_split(blurred_images, original_images, test_size=0.2, random_state=42)

# Trenowanie modelu
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=8)

# Zapisywanie modelu
model.save('deblur_cnn_model.h5')

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 70s 12s/step - loss: 0.0820 - val_loss: 0.0799
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 61s 10s/step - loss: 0.0721 - val_loss: 0.0559
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 69s 12s/step - loss: 0.0527 - val_loss: 0.0400
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 83s 12s/step - loss: 0.0337 - val_loss: 0.0218
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 71s 10s/step - loss: 0.0186 - val_loss: 0.0139
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 80s 10s/step - loss: 0.0143 - val_loss: 0.0102
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 55s 9s/step - loss: 0.0109 - val_loss: 0.0098
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 83s 9s/step - loss: 0.0097 - val_loss: 0.0094
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 60s 10s/step - loss: 0.0095 - val_loss: 0.0088
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 80s 10s/step - loss: 0.0088 - val_loss: 0.0077
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 82s 10s/step - loss: 0.0077 - val_loss: 0.0072
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 61s 10s/step - loss: 0.0070 - val_loss: 0.0081
Epo

In [18]:
restored_image = model.predict(np.expand_dims(X_val[2], axis=0))

restored_image = restored_image[0] * 255
restored_image = restored_image.astype('uint8')

restored_image_pil = Image.fromarray(restored_image)
restored_image_pil.save('restored2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
